In [2]:
# define a state class such that it can be used to measure some or all qubits, evolve a random brick work circuit
# compute expectation value of a given Pauli string
# apply a specific gate on it, could be supplied as a full unitary or 
# state attributes : norm, full array
# state.measure([Z Z Z])
# state.bwm_evolve(n_steps):
# state.apply(gate)
# state.reset()
# state.compute_ee( ) # entanglement entropy

In [115]:
import numpy as np
from itertools import product
from scipy.stats import unitary_group

X=np.array([[0,1],[1,0]],dtype=complex)
Y=np.array([[0,-1j],[1j,0]],dtype=complex)
Z=np.array([[1,0],[0,-1]],dtype=complex)

class qstate:
    def __init__(self,n=1,random=False,arr=[]):
        
        self.nqubits=n
        
        if(random==True):
            self.arr=np.random.rand(2**n)+1j*np.random.rand(2**n)
            self.arr=self.arr/np.linalg.norm(self.arr)
            
        elif(random==False and len(arr)==0):
            self.arr=np.zeros(2**n,dtype=complex)
            self.arr[0]=1
            
        elif(len(arr)!=0):
            self.arr=np.array(arr,dtype=complex)/np.linalg.norm(arr)
            self.nqubits=int(np.log2(len(arr)))
            
        self.norm=np.linalg.norm(self.arr)
    
    
    def apply_1gate(self,gate,i=0,inplace=True):

        #apply gate on qubit i (0 indexed)
        
        Ntot=2**self.nqubits # total dimesnions of the Hilbert space

        arr=self.arr.copy()
        gate=np.array(gate,dtype=complex)
        
        # say if m ranges from 0 to Ntot-1, mp ranges from 0 to Ntot/2 -1 since ith position bit string is fixed

        L=self.nqubits
        
        alpha,beta=0,1 # two states which can be generalized for qudits later
        
        for mp in range(Ntot//2):

            # suppose mp = m1'+m2', where m1'=b1x2**(L-2)+b2x2**(L-3)+... +b_{i}x2**(L-i-1), m2'=b_{i+2}2**(L-i-2)+...+b_{L-1}2**(1)+b_{L}q**(0)
            # then by definition m1' = (2**(L-i-1))*mp//2**(L-i-1)
            # and m2'=mp-m1'
            m1p=(2**(L-i-1))*(mp//(2**(L-i-1)))
            m2p=mp-m1p

            # now the two states which will be modified are ones which have alpha and beta at ith position

            alpha_ind=2*m1p+alpha*(2**(L-i-1))+m2p

            beta_ind=2*m1p+beta*(2**(L-i-1))+m2p

            # modifying the state at the corresponding indices

            arr[alpha_ind]=self.arr[alpha_ind]*gate[0,0]+gate[0,1]*self.arr[beta_ind]
            arr[beta_ind]=self.arr[beta_ind]*gate[1,1]+gate[1,0]*self.arr[alpha_ind]

        if(inplace==True):
            self.arr=arr
            self.norm=np.linalg.norm(self.arr)
            return None
        else:
            return arr
    
    def apply_2gate(self,gate,i=0,j=1,inplace=True):
        
        if(self.nqubits<2):
            return print("Number of qubits smaller than 2!")
            #raise ValueError("Number of qubits smaller than 2!")
            
                
        # pick four levels at the two sites, for qubits by default 0 and 1
        a1,b1= 0,1
        a2,b2= 0,1


        Ntot=2**self.nqubits # total dimesnions of the Hilbert space

        arr=self.arr.copy()
        gate=np.array(gate,dtype=complex)

        # say if m ranges from 0 to Ntot-1, mp ranges from 0 to Ntot/q**2 -1 since ith and jth position dit string is fixed
        # important that i<j!

        if(i>j):
            tmp=i
            i=j
            j=tmp

        q=2 # for qubits
        L=self.nqubits 
        
        for mp in range(Ntot//(q**2)):

            # suppose mp = m1'+m2'+m3', 
            # where m1'=b1q**(L-3)+b2q**(L-4)+... +b_{i-1}q**(L-i-1), m2'=b_{i+1}q**(L-i-2)+ ...+b_{j-1}q**(L-j)
            # ,m3'=b_{j+1}q**(L-j-1).....+b_{L-1}q**(1)+b_{L}q**(0)
            # then by definition m1' = (q**(L-i-1))*mp//q**(L-i-1)
            # and m2'=(q**(L-j))*mp//q**(L-j)-m1'
            # and m3'= m'-m1'-m2'

            m1p=(q**(L-i-2))*(mp//(q**(L-i-2)))
            m2p=(q**(L-j-1))*(mp//(q**(L-j-1)))-m1p
            m3p=mp-m1p-m2p

            # now the four states which will be modified are ones which have alpha and beta at ith position

            a1a2_ind=(q**2)*m1p+a1*(q**(L-i-1))+q*m2p+a2*(q**(L-j-1))+m3p
            a1b2_ind=(q**2)*m1p+a1*(q**(L-i-1))+q*m2p+b2*(q**(L-j-1))+m3p            
            b1a2_ind=(q**2)*m1p+b1*(q**(L-i-1))+q*m2p+a2*(q**(L-j-1))+m3p
            b1b2_ind=(q**2)*m1p+b1*(q**(L-i-1))+q*m2p+b2*(q**(L-j-1))+m3p

            
            psi_short=np.array([self.arr[a1a2_ind],self.arr[a1b2_ind],self.arr[b1a2_ind],self.arr[b1b2_ind]])
            
            arr[a1a2_ind]=gate[0,:].dot(psi_short)
            arr[a1b2_ind]=gate[1,:].dot(psi_short)
            arr[b1a2_ind]=gate[2,:].dot(psi_short)
            arr[b1b2_ind]=gate[3,:].dot(psi_short)        

        if(inplace==True):
            self.arr=arr
            self.norm=np.linalg.norm(self.arr)
            return None
        else:
            return arr

        
        
        
    def measure(self,basis_list=['Z'],only_subsystem=False):
        

        if(basis_list==['I']*self.nqubits): #if no qubits are being measured
            return None
        
        
        # if non-standard, assume all qubits are being measured in Z basis
        if(len(basis_list)!=self.nqubits):
            basis_list=['Z']*self.nqubits
        
        
        # if basis element 'I', qubit is NOT measured
        meas_qubits=[]

        for i in range(self.nqubits):
            
            if(basis_list[i]=='X'):
                self.arr.apply_1gate(X,i)
                meas_qubits.append(i)
            elif(basis_list[i]=='Y'):
                self.arr.apply_1gate(Y,i)
                meas_qubits.append(i+1)
                
            elif(basis_list[i]=='Z'):
                meas_qubits.append(i)
                
        all_qubits=[i for i in range(self.nqubits)]
        
        
        n_meas=len(meas_qubits)

        traced_qubits=np.setdiff1d(np.arange(self.nqubits),meas_qubits)
        
        shape_arr=np.repeat(2,self.nqubits)
        psi=np.reshape(self.arr,shape_arr)

        if(n_meas!=self.nqubits):
            prob_arr=np.sum(np.abs(psi)**2,axis=tuple(traced_qubits)) #tracing out qubits that won't be measured
        
        else:
            prob_arr=np.abs(psi)**2

        # reshape to sample integer
        prob_arr=np.reshape(prob_arr,2**n_meas)

        meas_int=np.random.choice(range(2**n_meas),p=prob_arr) #choosing the computational basis state randomly after measurement based on 
        meas_str = format(meas_int,'0'+str(n_meas)+'b')

        
        # initialising collapsed state on the unmeasured system
        if(only_subsystem):
            psi_b=np.zeros(2**(self.nqubits-n_meas),dtype="complex")

        else:
            psi_b=np.zeros(2**self.nqubits,dtype="complex")
            

        # find the full collapsed state including the unmeasured qubits
        for i in range(2**(self.nqubits-n_meas)):
            
            # find the full binary string inserting the measured string
            bin_i=format(i,'0'+str(self.nqubits-n_meas)+'b')
            bin_full=['0']*self.nqubits
            
            for j,m in enumerate(meas_qubits):
                bin_full[m]=meas_str[j]#str(meas_ind[j])
                
            for j,m in enumerate(traced_qubits):
                bin_full[m]=str(bin_i[j])

            new_i=int("".join(bin_full),2)

            if(only_subsystem):
                psi_b[i]=self.arr[new_i]
                
            else:
                psi_b[new_i]=self.arr[new_i]
                
            
        
        # update number of qubits and state array
        if(only_subsystem):
            self.nqubits=self.nqubits-n_meas
                             
        self.arr=psi_b/np.linalg.norm(psi_b) # normalize                    
        
        return meas_qubits,meas_str
    
    
    def compute_ent_ee(self,subsystem_size = None,svd_thresh=1e-8) :
        
        if(subsystem_size==None):
            subsystem_size=self.nqubits//2
        
        # reshape psi so singular values can be computed
        psi=np.reshape(self.arr,(2**(subsystem_size),2**(self.nqubits-subsystem_size)))
        
        s=np.linalg.svd(psi,compute_uv=False)

        ent_entropy = 0
        for j in range(len(s)):
            if(s[j]>svd_thresh): # ignoring very small values
                ent_entropy=ent_entropy-np.log(s[j]**2)*(s[j]**2)
    
    
        return ent_entropy
    
    

    def random_brickwork_evolve(self,nsteps=1,pbc=True,p_meas=None):
        #periodic boundary conditions by default
        
        if(pbc==True and self.nqubits%2!=0):
            return "For periodic boundary conditions, need an even number of qubits"
        
        
        for step in range(nsteps):
        
            # layer 1 for even numbered qubits
            
            for m in range(0,self.nqubits,2):
                
                twogate=unitary_group.rvs(4)
                self.apply_2gate(gate=twogate,i=m,j=m+1)
                
            # layer 2 for odd numbered qubits
            
            for m in range(1,self.nqubits-1,2):
                
                twogate=unitary_group.rvs(4)
                self.apply_2gate(gate=twogate,i=m,j=m+1)            
        
            # if pbc add a gate between first and last in layer 2
            if(pbc==True and self.nqubits>2):
                twogate=unitary_group.rvs(4)
                self.apply_2gate(gate=twogate,i=0,j=self.nqubits-1)
                
                
            
            # measure the state after each layer with a probability p_meas
            if(p_meas != None):
                meas_basis=[]
                for i in range(self.nqubits):
                    if(np.random.rand()<p_meas):
                        meas_basis.append('Z')
                        
                    else:
                        meas_basis.append('I')
                        
                self.measure(meas_basis)

        return None

In [83]:
# check partial measurement results

state1=qstate(n=5,random=True)
print(state1.arr)
print("number of qubits",state1.nqubits)

X=np.array([[0,1],[1,0]])
#state1.apply_1gate(X,inplace=False)
#print('state after')
#print(state1.arr)
print("measured state from return ",state1.measure(['Z','I','Z','I','I'],only_subsystem=False))
print("state after collapse",state1.arr)

[0.16526777+0.16093762j 0.19062439+0.07568335j 0.15280562+0.05084543j
 0.01296345+0.12600014j 0.0480168 +0.09784073j 0.04333499+0.18975034j
 0.1026039 +0.10845845j 0.11572788+0.06778217j 0.01399153+0.19306042j
 0.00866566+0.04942486j 0.01242422+0.13274783j 0.20337611+0.09795778j
 0.15755689+0.06827274j 0.14268695+0.12384003j 0.05486834+0.04859653j
 0.18521033+0.07927472j 0.03190272+0.12448993j 0.13009742+0.03878247j
 0.16998108+0.13439977j 0.20279878+0.04450161j 0.04765491+0.18530998j
 0.14141909+0.10341813j 0.04667632+0.17863204j 0.1152046 +0.07429377j
 0.15760883+0.13183118j 0.11851175+0.1892154j  0.18919021+0.06999445j
 0.18679551+0.04714605j 0.18919296+0.15564637j 0.12404556+0.06038536j
 0.15916427+0.08952441j 0.01182742+0.15883885j]
number of qubits 5
traced [1 3 4]
measured state from return  ([0, 2], '11')
state after collapse [0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.    

In [13]:
# check computation of entanglement entropy

#psi=qstate(n=2,random=True)
psi=qstate(arr=[0.5,0,0,0.5])
print(psi.arr)
print("entanglement entropy")
print(psi.compute_ent_ee())

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
entanglement entropy
0.6931471805599454


In [77]:

#gate=np.eye(4)

#cuegate=cue(4)
cuegate=unitary_group.rvs(4)
#gate2=np.kron(np.kron(np.eye(2),cuegate),np.eye(2))
gate2=np.kron(np.eye(4),cuegate)



psi=qstate(n=4,random=True)
print("psi before",psi.arr)

manual_mult_psi=gate2.dot(psi.arr)
print("manual")
print(manual_mult_psi)
psi.apply_2gate(gate=cuegate,i=2,j=3,inplace=True)
print("psi after")

print(psi.arr)

print("difference between manual product and using function",np.linalg.norm(psi.arr-manual_mult_psi))

psi.apply_2gate(gate=np.conjugate(np.transpose(cuegate)),i=2,j=3,inplace=True)
print("psi after U dagger applied",psi.arr)



psi before [0.17518418+0.15038016j 0.14757404+0.25466807j 0.22067397+0.29369412j
 0.12158306+0.24618533j 0.24813397+0.09458458j 0.0842843 +0.0456501j
 0.15408354+0.17450809j 0.18372853+0.2054555j  0.01053782+0.08209335j
 0.14775005+0.03399659j 0.23055551+0.1594362j  0.2048137 +0.04897363j
 0.25840906+0.18066753j 0.13718127+0.05785644j 0.11835404+0.06036678j
 0.23036769+0.30787964j]
manual
[-0.29649103-0.24538985j -0.00435287-0.15329033j -0.06838263-0.19671808j
  0.3436835 +0.13095034j -0.22036314-0.15289111j -0.18404602-0.0190613j
 -0.09473474-0.1258588j   0.27452272+0.0593104j  -0.22058132-0.07789523j
 -0.03316185-0.12144254j -0.04357542+0.07766169j  0.27027516-0.03476226j
 -0.11488219-0.13794787j -0.26027269-0.10484908j -0.130884  -0.20296558j
  0.3422072 +0.02643651j]
indices are 0 1 2 3
indices are 4 5 6 7
indices are 8 9 10 11
indices are 12 13 14 15
psi after
[-0.29649103-0.24538985j -0.00435287-0.15329033j -0.06838263-0.19671808j
  0.3436835 +0.13095034j -0.22036314-0.15289111j 

In [123]:
# evolve BWM

psi=qstate(n=4,random=False)

print("EE before",psi.compute_ent_ee())

psi.random_brickwork_evolve(nsteps=100)

print("EE after",psi.compute_ent_ee())

psi_random=qstate(arr=unitary_group.rvs(2**4).dot(qstate(n=4,random=False).arr))
print("EE of Haar random",psi_random.compute_ent_ee())


EE before 0.0
EE after 0.9169515708966764
EE of Haar random 0.9964564492982337


In [114]:
# measure with probability p
psi=qstate(n=4,random=False)

print("EE before",psi.compute_ent_ee())

psi.random_brickwork_evolve(nsteps=100,p_meas=0.1)

print("EE after",psi.compute_ent_ee())


EE before 0.0
EE after 0.9456555010278388
